In [1]:
import gzip
import json
import os

from __future__ import print_function, unicode_literals

In [2]:
catalog_dir = os.path.join('..','catalogs')

In [3]:
files = os.listdir(catalog_dir)

In [4]:
files

[u'ecoinvent_3.2_apos_xlsx.json.gz',
 u'ecoinvent_3.2_consequential_xlsx.json.gz',
 u'ecoinvent_3.2_cut-off_xlsx.json.gz',
 u'ecoinvent_3.2_undefined_xlsx.json.gz',
 u'elcd_3.2.json.gz',
 u'gabi_2016_all-extensions.json.gz',
 u'gabi_2016_professional-database-2016.json.gz',
 u'uslci_ecospold.json.gz']

## Select databases of interest

In [5]:
my_files = [os.path.join(catalog_dir, files[k]) for k in (3, 4, 6, 5, 7)]  # we love list comprehensions!

In [6]:
my_files

[u'..\\catalogs\\ecoinvent_3.2_undefined_xlsx.json.gz',
 u'..\\catalogs\\elcd_3.2.json.gz',
 u'..\\catalogs\\gabi_2016_professional-database-2016.json.gz',
 u'..\\catalogs\\gabi_2016_all-extensions.json.gz',
 u'..\\catalogs\\uslci_ecospold.json.gz']

Give the databases short names for reference

In [7]:
names = ['EI (u)', 'ELCD', 'GaBi-Pro', 'GaBi-Ext', 'US LCI']

In [8]:
def load_archive(filename):
    with gzip.open(filename, 'r') as fp:
        J = json.load(fp)

    # GaBi datasets are collections of archives, whereas the others are just archives
    if 'archives' in J:
        return J['archives'][0]
    else:
        return J


Load all the archives into a collection called 'C'

In [9]:
C = [load_archive(fname) for fname in my_files]

(should take about 2-5 seconds)

## Create geography grid (Table 4 in manuscript)

In [10]:
geog = []
for i, archive in enumerate(C):
    for p in archive['processes']:
        geog.append({'db': names[i], 'process': p['tags']['Name'], 'geog': p['tags']['SpatialScope']})

there should be one entry in `geog` for each process listed in a database - total of around 25,000 processes

In [12]:
len(geog)

25287

use `pandas` to draw the pivot charts

In [13]:
import pandas as pd

In [14]:
P = pd.DataFrame(geog).pivot_table(index='geog', columns='db',  aggfunc=len, fill_value='', margins=True)

In [15]:
P.sort_values(by=('process','All'), ascending=False)[:20]  # only show the top 20 rows

process                                       
db     EI (u) ELCD GaBi-Ext GaBi-Pro US LCI      All
geog                                                
All     13307  503     7457     3319    701  25287.0
GLO      6218   25      446      338     15   7042.0
DE        168   19     2131      314          2632.0
US         92          1179      137     16   1424.0
RNA        13           649       19    667   1348.0
CH       1260   10       44       33          1347.0
RER      1136   75       14       84      3   1312.0
EU-27           96      296      869          1261.0
CA-QC     346                                  346.0
IN         60           187       55           302.0
IT         73   11      149       52           285.0
BR         66           153       59           278.0
NL         76   10      117       65           268.0
CN         65            99       99           263.0
FR         94   10      106       50           260.0
GB         70   10       97       62           239.0
AT         73   10       94       47           224.0
ES         75   10       75       34           194.0
SE         77   10       60       46           193.0
BE         67   10       72       44           193.0

## Count Reference Flow Frequency (table 3 in manuscript)

In [16]:
from collections import Counter

In [18]:
def create_flow_map(archive):
    """
    This function creates a hash map from the entityId to the entity's tags- very fast
    """
    flow_map = dict()
    for f in archive['flows']:
        flow_map[f['entityId']] = f['tags']
    return flow_map


In [19]:
def count_ref_flows(archive):
    rfs = Counter()
    flow_map = create_flow_map(archive)
    for i in archive['processes']:
        x = i['referenceExchange']
        if x == "None":
            count_key = (None, None)
        else:
            direc, flowref = x.split(': ')
            try:
                flowname = flow_map[flowref]['Name']
            except KeyError:
                flowname = flow_map[int(flowref)]['Name']
            except KeyError:
                flowname = 'Flow Not Found!'
            count_key = (direc, flowname)
        rfs[count_key] += 1
    return rfs


In [20]:
rf_count = []
for i, archive in enumerate(C):
    print('Parsing archive %s' % names[i])
    rfs = count_ref_flows(archive)
    for rf, count in rfs.items():
        try:
            rf_count.append({'db': names[i], 'exchange': '%s: %s' % (rf[0], rf[1]), 'count': count})
        except TypeError:
            print('rf: %s (type %s) count: %d' %(rf, type(rf), count))

Parsing archive EI (u)
Parsing archive ELCD
Parsing archive GaBi-Pro
Parsing archive GaBi-Ext
Parsing archive US LCI


(should take << 1 second)

In [21]:
RF = pd.DataFrame(rf_count).pivot_table(index='exchange', columns='db', aggfunc=sum, fill_value='', margins=True)

In [22]:
RF.sort_values(('count','All'), ascending=False)[:20]

count                \
db                                                 EI (u) ELCD GaBi-Ext   
exchange                                                                  
All                                                 13307  503     7457   
Output: electricity, high voltage                    1974                 
Output: Thermal energy (MJ)                                         944   
Output: Electricity                                         64      522   
Output: Steam (MJ)                                                  340   
Output: electricity, low voltage                      730                 
Input: Housing technology                                           340   
Output: electricity, medium voltage                   423                 
Output: heat, district or industrial, other tha...    300                 
Output: Cargo                                                        80   
Output: heat, district or industrial, natural gas     141                 
None: None                                                   3      107   
Input: Aluminium scrap                                               60   
Input: Incineration good                                             53   
Input: Steel scrap                                                   55   
Output: blast furnace gas                             108                 
Output: biogas                                         97                 
Output: Elevator utilization                                         90   
Output: Used cooling (Emission air conditioner)                      86   
Output: Hour of operation                                            84   

                                                                             
db                                                 GaBi-Pro US LCI      All  
exchange                                                                     
All                                                    3319    701  25287.0  
Output: electricity, high voltage                                    1974.0  
Output: Thermal energy (MJ)                             236          1180.0  
Output: Electricity                                     471          1057.0  
Output: Steam (MJ)                                      622           962.0  
Output: electricity, low voltage                                      730.0  
Input: Housing technology                               191           531.0  
Output: electricity, medium voltage                                   423.0  
Output: heat, district or industrial, other tha...                    300.0  
Output: Cargo                                           127           207.0  
Output: heat, district or industrial, natural gas                     141.0  
None: None                                               10      1    121.0  
Input: Aluminium scrap                                   59           119.0  
Input: Incineration good                                 58           111.0  
Input: Steel scrap                                       54           109.0  
Output: blast furnace gas                                             108.0  
Output: biogas                                                         97.0  
Output: Elevator utilization                              6            96.0  
Output: Used cooling (Emission air conditioner)           4            90.0  
Output: Hour of operation                                 3            87.0

## Text Co-occurrence (table 5 in manuscript)